In [2]:
import re

def iteritems(dic):
    return iter(dic.items())

def transliterate(string):

    capital_letters = {u'А': u'A',
                       u'Б': u'B',
                       u'В': u'V',
                       u'Г': u'G',
                       u'Д': u'D',
                       u'Е': u'E',
                       u'Ё': u'E',
                       u'З': u'Z',
                       u'И': u'I',
                       u'Й': u'Y',
                       u'К': u'K',
                       u'Л': u'L',
                       u'М': u'M',
                       u'Н': u'N',
                       u'О': u'O',
                       u'П': u'P',
                       u'Р': u'R',
                       u'С': u'S',
                       u'Т': u'T',
                       u'У': u'U',
                       u'Ф': u'F',
                       u'Х': u'H',
                       u'Ъ': u'',
                       u'Ы': u'Y',
                       u'Ь': u'',
                       u'Э': u'E',}

    capital_letters_transliterated_to_multiple_letters = {u'Ж': u'Zh',
                                                          u'Ц': u'Ts',
                                                          u'Ч': u'Ch',
                                                          u'Ш': u'Sh',
                                                          u'Щ': u'Sch',
                                                          u'Ю': u'Yu',
                                                          u'Я': u'Ya',}


    lower_case_letters = {u'а': u'a',
                       u'б': u'b',
                       u'в': u'v',
                       u'г': u'g',
                       u'д': u'd',
                       u'е': u'e',
                       u'ё': u'e',
                       u'ж': u'zh',
                       u'з': u'z',
                       u'и': u'i',
                       u'й': u'y',
                       u'к': u'k',
                       u'л': u'l',
                       u'м': u'm',
                       u'н': u'n',
                       u'о': u'o',
                       u'п': u'p',
                       u'р': u'r',
                       u'с': u's',
                       u'т': u't',
                       u'у': u'u',
                       u'ф': u'f',
                       u'х': u'h',
                       u'ц': u'ts',
                       u'ч': u'ch',
                       u'ш': u'sh',
                       u'щ': u'sch',
                       u'ъ': u'',
                       u'ы': u'y',
                       u'ь': u'',
                       u'э': u'e',
                       u'ю': u'yu',
                       u'я': u'ya',}

    capital_and_lower_case_letter_pairs = {}

    for capital_letter, capital_letter_translit in iteritems(capital_letters_transliterated_to_multiple_letters):
        for lowercase_letter, lowercase_letter_translit in iteritems(lower_case_letters):
            capital_and_lower_case_letter_pairs[u"%s%s" % (capital_letter, lowercase_letter)] = u"%s%s" % (capital_letter_translit, lowercase_letter_translit)

    for dictionary in (capital_and_lower_case_letter_pairs, capital_letters, lower_case_letters):

        for cyrillic_string, latin_string in iteritems(dictionary):
            string = re.sub(cyrillic_string, latin_string, string)

    for cyrillic_string, latin_string in iteritems(capital_letters_transliterated_to_multiple_letters):
        string = re.sub(cyrillic_string, latin_string.upper(), string)

    return string

In [3]:
#Создаю поисковой запрос, после этого выкачиваю все резюме с первой страницы (надо узнать сколько всего страниц и выкачать также)
#после этого узнаю чему равна зарплата в каждом конкретном резюме и выкачиваю описание
string = "продавец-консультант"

In [4]:
def get_url(string):
    return "https://hh.ru/resumes/" + transliterate(string)

In [5]:
import requests
from bs4 import BeautifulSoup

In [6]:
headers = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36"}

r = requests.get(get_url(string), headers=headers)
r.ok
print(get_url(string))

https://hh.ru/resumes/prodavets-konsultant


In [7]:
r

<Response [200]>

In [8]:
soup = BeautifulSoup(r.text)

In [9]:
soup1 = soup.body.find_all('div', {"class" : "HH-MainContent HH-Supernova-MainContent"})[0].div.div

In [10]:
soup2 = soup1.find_all('div', {"id" : "HH-React-Root"})[0].div.div.div.find_all('div', {"class" : "resume-serp"})[0]

In [11]:
soup3 = soup2.find_all('div', {"class" : "bloko-column bloko-column_m-9 bloko-column_l-13"})[0].div

In [12]:
#Тут уже лежит список всех найденных страниц
soup4 = soup3.find_all('div', {'data-qa':"resume-serp__results-search"})[0].find_all('div', {'data-qa':"resume-serp__resume"})

In [13]:
def get_local_url(soup):
    return 'https://hh.ru/' + soup[0].find_all('div')[1].div.div.div.span.a.get('href')

In [14]:
#тут мы нашли нужные url. Для нахождения остальных надо к изначальному урл приписать страницу
new_urls = []
for tap in soup4:
    new_urls.append('https://hh.ru' + tap.find_all('div')[1].div.div.div.span.a.get('href'))

In [15]:
new_urls[0]

'https://hh.ru/resume/085211a50002f385090039ed1f72615a374c66?query=%D0%BF%D1%80%D0%BE%D0%B4%D0%B0%D0%B2%D0%B5%D1%86-%D0%BA%D0%BE%D0%BD%D1%81%D1%83%D0%BB%D1%8C%D1%82%D0%B0%D0%BD%D1%82&hhtmFrom=resumes_catalog&source=search'

In [32]:
headers = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36"}

r2 = requests.get(new_urls[0], headers=headers)
r2.ok
soup5 = BeautifulSoup(r2.text)
print(new_urls[0])

https://hh.ru/resume/085211a50002f385090039ed1f72615a374c66?query=%D0%BF%D1%80%D0%BE%D0%B4%D0%B0%D0%B2%D0%B5%D1%86-%D0%BA%D0%BE%D0%BD%D1%81%D1%83%D0%BB%D1%8C%D1%82%D0%B0%D0%BD%D1%82&hhtmFrom=resumes_catalog&source=search


In [17]:
#body > div.HH-MainContent.HH-Supernova-MainContent
#HH-React-Root > div > div.bloko-columns-wrapper > div > div > div > div.resume-wrapper > div > div
#HH-React-Root > div > div.bloko-columns-wrapper > div > div > div > div.resume-wrapper > div > div > 
#div.bloko-gap.bloko-gap_top > div:nth-child(1) > div.bloko-columns-row > 
#div.bloko-column.bloko-column_xs-4.bloko-column_s-3.bloko-column_m-3.bloko-column_l-3 > div > h2 > span

In [18]:
soup6 = soup5.body.find_all('div', {'class': 'HH-MainContent HH-Supernova-MainContent'})[0].div.div

In [19]:
soup7 = soup6.find_all('div', {'id': 'HH-React-Root'})[0]

In [33]:
soup5.findAll("span", {"class":"resume-block__salary resume-block__title-text_salary"})[0].text

'50\u2009000\xa0руб.'

In [34]:
def get_salary(url):
    headers = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36"}
    r2 = requests.get(url, headers=headers)
    if r2.ok:
        soup5 = BeautifulSoup(r2.text)
        if len(soup5.findAll("span", {"class":"resume-block__salary resume-block__title-text_salary"})) > 0:
            print(soup5.findAll("span", {"class":"resume-block__salary resume-block__title-text_salary"})[0].text)
            return(soup5.findAll("span", {"class":"resume-block__salary resume-block__title-text_salary"})[0].text)
        else:
            print('There no salary')
    else:
        print('There no such url')
        print(url)
        return '0'

In [35]:
get_salary(new_urls[6])

40 000 руб.


'40\u2009000\xa0руб.'

In [36]:
headers = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36"}

url = new_urls[0]
r2 = requests.get(url, headers=headers)
soup5 = BeautifulSoup(r2.text)

In [55]:
#массив описаний с предыдущих работ. Тут таже не хватает одного вида варианта блоков
soup5.findAll('div', {'data-qa' : 'resume-block-experience-description'})[0].text

'Консультация и продажа товаров Redmond ,работа на кассе . Прием и выкладка товара, ведение документации в 1С.'

In [38]:
#предыдущие места работы. Тут есть что-то лишнее, что обозначается одинаково, но при этом относится к 1 месту работы
len(soup5.findAll('div', {'class' : 'resume-block__sub-title'}))

18

In [56]:
#div.bloko-column.bloko-column_xs-4.bloko-column_s-2.bloko-column_m-2.bloko-column_l-2

In [39]:
#время работы на каждой отдельной. Это лучше всего показывает количество работ (предыдущие 2 считат либо мало, либо много)
soup5.findAll('div', {'class' : 'bloko-column bloko-column_xs-4 bloko-column_s-2 bloko-column_m-2 bloko-column_l-2'})[0].text

'Февраль\xa02020 — по\xa0настоящее\xa0время6\xa0месяцев'

In [99]:
#тут заголовки. первый заголовок это время работы
soup5.findAll('span', {'class' : 'resume-block__title-text resume-block__title-text_sub'})
len(soup5.findAll('div', {'data-qa' : 'resume-block-experience'})[0].findAll('div', {'class' : 'bloko-column bloko-column_xs-4 bloko-column_s-2 bloko-column_m-2 bloko-column_l-2'}))

8

In [52]:
#собираем ключевые навыки
key_skills = []
for s in soup5.findAll('div', {'data-qa' : 'bloko-tag bloko-tag_inline'}):
    print(s.span.text)
    key_skills.append(s.span.text)

Работа с большим объемом информации
Грамотная речь
Учет товаров
Проведение инвентаризаций
Торговля
оформление заказов


In [54]:
#суть в том, что у многих часть из этих разделом может быть пустой, поэтому надо сначала проверять, что этот раздел есть, а потом нем что-то искать
#также надо просто определить, что из этого нам надо, чтобы эту инфу собирать
soup5.findAll('span', {'class' : 'resume-block__title-text resume-block__title-text_sub'})

[<span class="resume-block__title-text resume-block__title-text_sub">Опыт работы <!-- -->17<!-- --> <!-- -->лет<!-- --> <!-- -->10<!-- --> <!-- -->месяцев</span>,
 <span class="resume-block__title-text resume-block__title-text_sub">Ключевые навыки</span>,
 <span class="resume-block__title-text resume-block__title-text_sub">Обо мне</span>,
 <span class="resume-block__title-text resume-block__title-text_sub">Среднее специальное образование</span>,
 <span class="resume-block__title-text resume-block__title-text_sub">Знание языков</span>,
 <span class="resume-block__title-text resume-block__title-text_sub">Повышение квалификации, курсы</span>,
 <span class="resume-block__title-text resume-block__title-text_sub">Гражданство, время в пути до работы</span>]

In [122]:
def get_info(url):
    headers = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36"}
    r2 = requests.get(url, headers=headers)
    soup5 = BeautifulSoup(r2.text)
    
    #массив описаний с предыдущих работ. Тут таже не хватает одного вида варианта блоков
    desc_past_job = []
    for tag in soup5.findAll('div', {'data-qa' : 'resume-block-experience-description'}):
        desc_past_job.append(tag.text)
    time_past_job = []
    #время работы на каждой отдельной. Это лучше всего показывает количество работ (предыдущие считат либо мало, либо много)
    #надо преобразовать в нормальный вид
    for tag in soup5.findAll('div', {'data-qa' : 'resume-block-experience'})[0].findAll('div', {'class' : 'bloko-column bloko-column_xs-4 bloko-column_s-2 bloko-column_m-2 bloko-column_l-2'}):
        time_past_job.append(tag.text)
        print(time_past_job[-1])
    #количество предыдущих работ
    num_job = len(time_past_job)
    print('Количество предыдущих работ ' + str(num_job))
    print('')
    #собираем ключевые навыки
    key_skills = []
    print("Ключевые навки")
    for s in soup5.findAll('div', {'data-qa' : 'bloko-tag bloko-tag_inline'}):
        print(s.span.text)
        key_skills.append(s.span.text)
    #собираем образование
    #тут пока все вместе и курсы и повышение квалификации. Чтобы разделить надо найдо соупы, где они разделяются
    #также тут в обоих случаях есть тайтл и организация/описание
    education = []
    print('')
    print('Образование')
    for tag in soup5.findAll('div', {'data-qa' : 'resume-block-education-item'}):
        title = tag.div.text
        if (len(tag.find_all('div', {'data-qa': 'resume-block-education-organization'}))>0):
            organisation = tag.find_all('div', {'data-qa': 'resume-block-education-organization'})[0].text
        else:
            organisation = ''
        education.append((title, organisation))
        print(education[-1])
    

In [88]:
#тут пока все вместе и курсы и повышение квалификации. Чтобы разделить надо найдо соупы, где они разделяются
#также тут в обоих случаях есть тайтл и описание, их можно разделить, первй лежит в .div
soup5.findAll('div', {'data-qa' : 'resume-block-education-item'})[1].find_all('div', {'data-qa': 'resume-block-education-organization'})[0].text

'УКЦ "Престиж" г.Ялта, Менеджер туризма и гостиничного бизнеса'

In [102]:
get_info(new_urls[0])

Февраль 2020 — по настоящее время6 месяцев
Сентябрь 2018 — Февраль 20201 год 6 месяцев
Сентябрь 2017 — Март 20187 месяцев
Март 2016 — Март 20171 год 1 месяц
Октябрь 2015 — Февраль 20165 месяцев
Июль 2011 — Февраль 20142 года 8 месяцев
Май 2004 — Июнь 20117 лет 2 месяца
Май 2000 — Май 20044 года 1 месяц
Количество предыдущих работ 8

Ключевые навки
Работа с большим объемом информации
Грамотная речь
Учет товаров
Проведение инвентаризаций
Торговля
оформление заказов

Образование
('муз.училище им.П.И.Чайковского г.Симферополь', 'струнный отдел, артист оркестра,преподаватель детских музыкальных школ')
('Менеджмент туризма и гостиничного бизнеса', 'УКЦ "Престиж" г.Ялта, Менеджер туризма и гостиничного бизнеса')


In [123]:
for url in new_urls:
    print('')
    print('SEPARATE')
    print(url)
    print('')
    get_info(url)


SEPARATE
https://hh.ru/resume/085211a50002f385090039ed1f72615a374c66?query=%D0%BF%D1%80%D0%BE%D0%B4%D0%B0%D0%B2%D0%B5%D1%86-%D0%BA%D0%BE%D0%BD%D1%81%D1%83%D0%BB%D1%8C%D1%82%D0%B0%D0%BD%D1%82&hhtmFrom=resumes_catalog&source=search

Февраль 2020 — по настоящее время6 месяцев
Сентябрь 2018 — Февраль 20201 год 6 месяцев
Сентябрь 2017 — Март 20187 месяцев
Март 2016 — Март 20171 год 1 месяц
Октябрь 2015 — Февраль 20165 месяцев
Июль 2011 — Февраль 20142 года 8 месяцев
Май 2004 — Июнь 20117 лет 2 месяца
Май 2000 — Май 20044 года 1 месяц
Количество предыдущих работ 8

Ключевые навки
Работа с большим объемом информации
Грамотная речь
Учет товаров
Проведение инвентаризаций
Торговля
оформление заказов

Образование
('муз.училище им.П.И.Чайковского г.Симферополь', 'струнный отдел, артист оркестра,преподаватель детских музыкальных школ')
('Менеджмент туризма и гостиничного бизнеса', 'УКЦ "Престиж" г.Ялта, Менеджер туризма и гостиничного бизнеса')

SEPARATE
https://hh.ru/resume/6cfacd3b0007af3dc10039

Апрель 2017 — по настоящее время3 года 4 месяца
Июнь 2013 — Октябрь 20152 года 5 месяцев
Август 2012 — Июнь 201311 месяцев
Январь 2010 — Июнь 20122 года 6 месяцев
Июнь 2005 — Сентябрь 20094 года 4 месяца
Количество предыдущих работ 5

Ключевые навки
Пользователь ПК
Подготовка коммерческих предложений
Водительское удостоверение категории B
1С: Торговля и склад
Консультативные продажи
Прямые продажи
Ориентация на результат
Ведение переговоров
Организаторские навыки
снабжение
Знание устройства автомобиля

Образование
('ФГБО УВПО МГУ им. Н.П ОГАРЕВА', 'экономический, Финансы и кредит')
('Темниковский сельско хозяйственный колледж', 'Бухгалтерский, бухгалтер')
('ЧОУ ДПО "Саранский Дом науки и техники РСНИИОО"', 'ЧОУ ДПО "Саранский Дом науки и техники РСНИИОО", "Специалист, ответственный за обеспечение безопасности дорожного движения"')
('Минестерство транспорта РФ', 'Федеральная служба по надзору в сфере транспорта, Прошел обучение по обеспечению безопасности дорожного движения на автомобил

In [117]:
url = 'https://hh.ru/resume/305de5ae000723b9390039ed1f4b4c41716c6d?query=продавец-консультант&hhtmFrom=resumes_catalog&source=search'

In [118]:
headers = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36"}

r2 = requests.get(url, headers=headers)
soup5 = BeautifulSoup(r2.text)

In [119]:
soup5.findAll('div', {'data-qa' : 'resume-block-education-item'})[0].find_all('div', {'data-qa': 'resume-block-education-organization'})[0].text

'Государственное и муниципальное управление, Управление и связь с общественностью '

In [120]:
get_info(url)

Октябрь 2017 — Июнь 20191 год 9 месяцев
Декабрь 2016 — Сентябрь 201710 месяцев
Июль 2014 — Сентябрь 20162 года 3 месяца
Июнь 2010 — Июнь 20144 года 1 месяц
Март 2009 — Май 20101 год 3 месяца
Количество предыдущих работ 5

Ключевые навки
Активные продажи
опыт ведения переговоров 
Оценка стоимости автомобиля
ПК опытный пользователь 
Ориентация на результат
Управление персоналом
Ответственность
Быстрая обучаемость

Образование
('Курский институт менеджмента, экономики и бизнеса, Курск', 'Государственное и муниципальное управление, Управление и связь с общественностью ')
